In [7]:
import torch
x = torch.arange(4.0)
x # 生成序列

tensor([0., 1., 2., 3.])

In [8]:
# 然后我们其实应该需要指定一个地方存储梯度，但是我们不会对每个参数求到时都分配空间。
# 这样太麻烦了。 我们只需要指定变量需要导数这个参数即可。
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
print(x.grad) # 默认值是None

None


In [9]:
# 现在计算y
y = 2 * torch.dot(x, x) # x*x点乘, 得到标量
print(y) # tensor(28., grad_fn=<MulBackward0>)
# 2 * (0 + 1 + 4 + 9)
# 得到的y是一个标量。

# 然后因为已经使用了requires_grad=True, 因此, 得到, 我们要计算梯度的回调函数。
# 隐式的计算图.

tensor(28., grad_fn=<MulBackward0>)


In [10]:
# 调用反向传播函数, 来自动计算y关于x的每个分量的梯度.
y.backward() # 反向传播，计算梯度。
x.grad # 这样就得到了x的梯度。
# y对x求导, 就是y = 2x^2
# 因此求导得到的结果就是: 4x
# 然后将4x这个结果反向传播给x.

# 在grad前面, 一定要有backward. 因为计算梯度是一件很贵的一件事情, 因此只有显示的指定我要计算梯度, 然后向后传播, 这样, pytorch才会计算梯度.

tensor([ 0.,  4.,  8., 12.])

In [11]:
x.grad == 4 * x # 默认对Tensor的所有操作, 都是针对元素的操作.

tensor([True, True, True, True])

In [14]:
# 默认情况下, PyTorch会累积梯度, 我们需要清除之前的所有值.
x.grad.zero_() # tensor([0., 0., 0., 0.])
# 函数后面加上_, 在PT中表示原地操作, 也就是这里我直接原地对x的grad进行清零了.
y = x.sum()
# y = x1 + x2 + ...
# 求导, y = 1 + 1 + ...
y.backward()
x.grad # tensor([1., 1., 1., 1.])

tensor([1., 1., 1., 1.])

In [32]:
# y不是一个标量的情况下:
# 深度学习中, 我们的目的不是计算微分矩阵, 而是批量中每个样本单独计算的偏导数之和.
x.grad.zero_()
y = x * x # 此时y不是一个标量, 是一个向量
print(y) # tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)
# y是一个向量, 如何求导.

# 我们很少对于向量进行求导, 大部分情况, 我们使用sum()求和, 再求导. 后面会讲在什么情况下, 会对y进行sum
# 等价于y.backward(torch.ones(len(x))
y.sum().backward()
x.grad

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)


tensor([0., 2., 4., 6.])

In [38]:
# 将某些计算移动到记录的计算图之外.
#
x.grad.zero_() # empty x grad
y = x*x # [0, 1, 4, 9]
u = y.detach() # tensor([0., 1., 4., 9.])
print(u) # 但是u的require_grad=False, 因此这里u就不再计算图中了, 而是一个常数. 那么
# 有时候, 我们想要切断一些分支的反向传播, 就是用detach()
# detach()返回一个新的tensor, 从当前计算图分离下来, 但是仍然指向原变量存放的位置. 同时, 该参数的require_grad=False. 也就是不保存梯度.
# 后续, 在想要保存网络中一些值的时候, 这是一种很有效的方式.

z = u * x # tensor([0., 1., 4., 9.]) * tensor([0., 1., 2., 3.])
# z就相当于 常数*x, 求导, 就等于常数, 因此, 就等于u
z.sum().backward()
# 可见, 实际上,
#
print(x.grad) # # tensor([0., 1., 4., 9.])
print(x.grad == u)

tensor([0., 1., 4., 9.])
tensor([0., 1., 4., 9.])
tensor([True, True, True, True])


In [34]:
# 此时, 同样, y还是x的导数, y=x^2
x.grad.zero_()
y.sum().backward() #
x.grad # tensor([0., 2., 4., 6.])

tensor([0., 2., 4., 6.])

In [54]:
# 即使构建函数的计算图需要通过python控制流, 我们依然可以计算得到变量的梯度
def f(a):
    b = a * 2
    while b.norm() < 1000: # 范数, 长度.
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c
a = torch.randn(size=(), requires_grad=True) # 返回一个符合标准正太分布的随机数.
print(a)

d = f(a) # 计算计算.
d.backward()

a.grad == d/a


tensor(-0.2006, requires_grad=True)


tensor(True)